### `auth_with_creds()`
This authentication method accepts a username and password and establishes an authentication session with EDL and ASF.
This is the same set of credentials you use to log into https://urs.earthdata.nasa.gov/home.

**NOTE:** Even if you have valid Earthdata credentials, you still need to **verify your account** and **specify your User Type/Study Area/Affliation** to be able to download ALOS data. If you are having trouble authenticating or download data, this might be part of the problem. If you have not set these parameters, go to https://urs.earthdata.nasa.gov/profile to do so.

In [12]:
import asf_search as asf
import getpass

session = asf.ASFSession()
username = input('Username:')
password = getpass.getpass('Password:')

try:
    user_pass_session = asf.ASFSession().auth_with_creds(username, password)
except asf.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

Username: max.zhan
Password: ········


Success!


***
## Downloading
[View this search in Vertex](https://search.asf.alaska.edu/#/?dataset=UAVSAR&productTypes=METADATA&resultsLoaded=true&zoom=8.090&center=-90.488,28.359&polygon=POLYGON((-91.97%2028.78,-88.85%2028.78,-88.85%2030.31,-91.97%2030.31,-91.97%2028.78)))
  
With authentication handled, we can now begin downloading products. First, we will need some search results to work with:

In [7]:
import shapely

# Defines an overlapping region which will be converted to a shapely Polygon
coordinates = [[
    [-121.174, 35.7869],
    [-120.549, 35.7869],
    [-120.549, 36.6517],
    [-121.174, 36.6517],
    [-121.174, 35.7869]
]]

# The start and end times to be queried
start_time = '2006-01-01T00:00:00Z'
end_time = '2011-12-01T00:00:00Z'

# Additional options to query
frame = 710
relativeOrbit = 220
processingLevel = asf.PRODUCT_TYPE.L1_0
maxResults = None


# Process the above arguments to make a query to ASF search, using the login credentials from the previous cell
coord_polygon = shapely.Polygon(coordinates[0])
aoi = str(coord_polygon)
print(coord_polygon)
opts = {
    'platform': asf.PLATFORM.ALOS,
    'intersectsWith': aoi,
    'start': start_time,
    'end': end_time,
    # Added options
    'frame': frame,
    'relativeOrbit': relativeOrbit,
    # The only usable polarization with INSAR seems to be HH, HH_HV crashes the script (might be a bug with ISCE3)
    'polarization': [asf.POLARIZATION.HH],#, asf.POLARIZATION.HH_HV],
    'processingLevel': processingLevel,
    'maxResults': maxResults,
}

# The results of the search as a large dictionary, or the IDs/URLs as a distinct list
results = asf.geo_search(**opts)
asf_ids = [x.properties['fileID'] for x in results]
asf_urls = [x.properties['url'] for x in results]

print(f'{len(results)} results found')
print(f'Found IDs:\n{asf_ids}\n==============================')
print(f'Found URLs:\n{asf_urls}\n==============================')

POLYGON ((-121.174 35.7869, -120.549 35.7869, -120.549 36.6517, -121.174 36.6517, -121.174 35.7869))
9 results found
Found IDs:
['ALPSRP274410710-L1.0', 'ALPSRP267700710-L1.0', 'ALPSRP220730710-L1.0', 'ALPSRP207310710-L1.0', 'ALPSRP167050710-L1.0', 'ALPSRP106660710-L1.0', 'ALPSRP099950710-L1.0', 'ALPSRP093240710-L1.0', 'ALPSRP059690710-L1.0']
Found URLs:
['https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP274410710-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP267700710-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP220730710-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP207310710-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP167050710-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP106660710-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP099950710-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP093240710-L1.0.zip', 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP059690710-L1.0.zip']


***
## Downloading Products
To download a single `ASFProduct`, simply call its `download()` method, passing in a previously-authenticated session, a path, and optionally a filename. If no filename is provided, the default is to use the filename of the product iself, as described in `properties['fileName']`.

The cell below downloads all products found within the `results` list, but the list can be iterated or reduced to download a specific subset of ALOS-1 data.

In [11]:
import os

# The maximum number of files to download
max_downloads = 1#math.inf
# The folder to download the ALOS data to
download_path = os.path.abspath('./data')
# The number of parallel processes to use to download
processes = 5

# This downloads the searched ALOS data according to the parameters above
os.makedirs(download_path, exist_ok=True)
results[:min(len(results), max_downloads)].download(path=download_path, session=user_pass_session, processes=processes)
files = [os.path.join(download_path, x) for x in os.listdir(download_path)]
print(files)

/home/jovyan/.local/envs/isce3_src/lib/python3.9/site-packages/asf_search/download/download.py:65: UserWarning: File already exists, skipping download: /home/jovyan/alos-to-insar/data/ALPSRP274410710-L1.0.zip
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')


['/home/jovyan/alos-to-insar/data/.ipynb_checkpoints', '/home/jovyan/alos-to-insar/data/ALPSRP274410710-L1.0.zip']
